In [1]:
import dabench as dab
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
import jax
from timeit import default_timer as timer
import pandas as pd
import jaxlib

from ray import train, tune
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch

# Define some parameters

In [2]:
year_in_timesteps = 4380
spinup_size = 5*year_in_timesteps
valid_size = round(year_in_timesteps/4)
transient_size = 1*year_in_timesteps
test_size = 1*year_in_timesteps

In [3]:
nr_steps = spinup_size + valid_size + transient_size + test_size
delta_t=7200
analysis_window = 6*delta_t
analysis_time_in_window = 3*delta_t
num_epochs = 25

# Function def: Backprop

Note: Since we're using raytune, we're separating this into a prep function and a run function

In [4]:

def prep_backprop_4dvar(system_dim_xy, nr_steps, spinup_size, valid_size, test_size, test_run, delta_t, 
                        sigma_bg_multiplier, sigma_obs_multiplier, random_seed):
    np_rng = np.random.default_rng(random_seed)
    jax.clear_backends()


    ### Nature Run
    nature_run = dab.data.PyQGJax(nx=system_dim_xy, ny=system_dim_xy, delta_t=delta_t, store_as_jax=True, random_seed=random_seed)

    nature_run.generate(n_steps=nr_steps) 
    nr_spinup, nr_valid, nr_transient_and_test = nature_run.split_train_valid_test(
        spinup_size, valid_size, transient_size + test_size)
    nr_transient, nr_test, _ = nr_transient_and_test.split_train_valid_test(transient_size, test_size, 0)

    if not test_run:
        nr_eval = nr_valid
    else:
        nr_eval = nr_test
        
    ### Observations
    obs_location_count = round(nature_run.system_dim/2)

    # First we need to calculate the per-variable SD for QGS model
    obs_sd_scale = 0.1
    per_variable_sd = np.std(nr_spinup.values, axis=0)
    obs_sd = 0.1*per_variable_sd

    obs_pyqg = dab.observer.Observer(
        nr_eval, # Data generator object
        time_indices = np.arange(0, nr_eval.time_dim, 3), # Observation every other timestep
        random_location_count = obs_location_count, # Probability of picking each location in l63.system_dim for random sampling
        error_bias = 0.0, # Mean for observation error, Gaussian/Normal distribution
        error_sd = obs_sd, # Standard deviation for observation error, Gaussian/Normal distribution
        random_seed=random_seed+test_run, # We can specify a random seed. Default is 99
        stationary_observers=True,
        store_as_jax=True
    )

    obs_vec_pyqg = obs_pyqg.observe()

    ### Model
    model_pyqg = dab.data.PyQGJax(nx=system_dim_xy, ny=system_dim_xy, store_as_jax=True, random_seed=random_seed)

    class PyQGModel(dab.model.Model):                                                                       
        """Defines model wrapper for Lorenz96 to test forecasting."""
        def forecast(self, state_vec, n_steps):
            gridded_values = state_vec.values.reshape(self.model_obj.original_dim)
            self.model_obj.generate(x0=gridded_values, n_steps=n_steps)
            new_vals = self.model_obj.values

            new_vec = dab.vector.StateVector(values=new_vals, store_as_jax=True)

            return new_vec

        def _forecast_x0(self, x0, n_steps):
            self.model_obj.generate(x0=x0.reshape(self.model_obj.original_dim), n_steps=n_steps)
            return self.model_obj.values

        def compute_tlm(self, state_vec, n_steps):
            x0 = state_vec.values
            return jax.jacrev(self._forecast_x0, argnums=0)(x0, n_steps), self._forecast_x0(x0, n_steps)

    fc_model = PyQGModel(model_obj=model_pyqg)

    
    ### Set up DA matrices    
    obs_total_size = int(obs_location_count*3)
    sigma_obs=sigma_obs_multiplier*np.tile(obs_sd[obs_vec_pyqg.location_indices[0]], 3)
    sigma_bg = sigma_bg_multiplier*obs_sd
    H = np.zeros((obs_location_count, nature_run.system_dim))
    H[np.arange(H.shape[0]), np.tile(obs_vec_pyqg.location_indices[0], 1)] = 1
    R = (sigma_obs**2)* np.identity(obs_total_size)
    B = (sigma_bg**2)*np.identity(nature_run.system_dim)
    Bsqrt = np.sqrt(B)

    ### Set up DA Cycler
    da_time_start = timer()
    
    dc = dab.dacycler.Var4DBackprop(
        system_dim=nature_run.system_dim,
        delta_t=nr_eval.delta_t,
        H=H,
        B=B,
        R=R,
        # These will be set later in run_backprop_4dvar
        num_epochs=None, 
        learning_rate=None,
        lr_decay=None,
        model_obj=fc_model,
        obs_window_indices=[0,3,6],
        steps_per_window=7, # 7 instead of 6 because inclusive of 0 and 6
        )
    
    ### Execute
    cur_tstep = 0
    x0_original = nr_eval.values[cur_tstep] + np_rng.normal(size=(nature_run.system_dim,), scale=sigma_bg)

    x0_sv = dab.vector.StateVector(
        values=x0_original,
        store_as_jax=True)
    
    start_time = nr_eval.times[cur_tstep]
    
    return dc, x0_sv, start_time, obs_vec_pyqg, nr_eval, sigma_obs

In [5]:

def run_backprop_4dvar(config, dc, x0_sv, start_time, obs_vec, nr_eval, sigma_obs, 
                       analysis_window, analysis_time_in_window,
                       num_epochs):
    dc.learning_rate = config['lr']
    dc.lr_decay = config['lr_decay']
    dc.num_epochs = num_epochs
    try: 
        out_statevec = dc.cycle(
            input_state = x0_sv,
            start_time = start_time,
            obs_vector = obs_vec,
            analysis_window=analysis_window,
            timesteps=int(nr_eval.time_dim/6) - 2,
            obs_error_sd=sigma_obs,
            analysis_time_in_window=analysis_time_in_window)

        rmse = np.sqrt(np.mean(np.square(nr_eval.values[:out_statevec.values.shape[0]] - out_statevec.values)))
        train.report({'rmse':rmse})
    except jaxlib.xla_extension.XlaRuntimeError:
        train.report({'rmse':999999})
    
    return 


# Validation: test some different learning rates and decays

### Tune

In [6]:
space = {
    "lr": hp.loguniform("lr", -40, -1),
    "lr_decay": hp.uniform("lr_decay", 0.1, 0.99),
}



In [7]:
%%time
all_results_df_list = []
system_dim_xy_list = [16, 20, 24, 32]
for system_dim_xy in system_dim_xy_list:
    tune_time_start = timer()

    random_seed = system_dim_xy
    # Run prep
    run_dict = dict(
        system_dim_xy=system_dim_xy, 
        nr_steps=nr_steps,
        spinup_size=spinup_size,
        valid_size=valid_size,
        test_size=test_size,
        test_run=False,
        delta_t=delta_t,
        sigma_bg_multiplier=0.5,
        sigma_obs_multiplier=1.25,
        random_seed=random_seed)
    dc, x0_sv, start_time, obs_vec, nr_eval, sigma_obs = prep_backprop_4dvar(**run_dict)
    
    
    print('Starting... {} system dim'.format(system_dim_xy))
    trainable_with_system_dim = tune.with_parameters(
        run_backprop_4dvar, 
        dc=dc,
        x0_sv=x0_sv,
        start_time=start_time,
        obs_vec=obs_vec,
        nr_eval=nr_eval,
        sigma_obs=sigma_obs, 
        analysis_window=analysis_window,
        analysis_time_in_window=analysis_time_in_window,
        num_epochs=num_epochs
    )
    trainable_with_resources = tune.with_resources(trainable_with_system_dim, {"cpu": 16})
    hyperopt_search = HyperOptSearch(space, metric="rmse", mode="min")
    tuner = tune.Tuner(
        trainable_with_system_dim,
        tune_config=tune.TuneConfig(
            num_samples=100,
            search_alg=hyperopt_search,
        ),
    )
    results = tuner.fit()

    cur_results_df = results.get_dataframe()

    cur_results_df['system_dim_xy'] = system_dim_xy
    
    tune_time = timer() - tune_time_start
    
    cur_results_df['total_tune_time'] = tune_time

    all_results_df_list.append(cur_results_df)

(run_backprop_4dvar pid=47464) No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(run_backprop_4dvar pid=47464) 2024-03-12 23:11:26.123610: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none
(run_backprop_4dvar pid=47464) 2024-03-12 23:11:26.502761: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none
(run_backprop_4dvar pid=47464) 2024-03-12 23:11:26.570286: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none
(run_backprop_4dvar pid=47464) 2024-03-12 23:11:29.327945: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none


(run_backprop_4dvar pid=47464) [[ 2211.70001013  2211.48763761  2211.28678234]
(run_backprop_4dvar pid=47464)  [ 2187.48863429  2187.27576011  2187.07440978]
(run_backprop_4dvar pid=47464)  [ 2149.35919536  2149.16486809  2148.98109078]
(run_backprop_4dvar pid=47464)  [ 2103.66750518  2103.46692688  2103.27723889]
(run_backprop_4dvar pid=47464)  [ 2233.60754174  2233.41819491  2233.239105  ]
(run_backprop_4dvar pid=47464)  [ 2245.05125045  2244.86710185  2244.69293498]
(run_backprop_4dvar pid=47464)  [ 2233.13466536  2232.93058943  2232.73756496]
(run_backprop_4dvar pid=47464)  [ 2249.45717493  2249.25973022  2249.07296825]
(run_backprop_4dvar pid=47464)  [ 2288.06489591  2287.85370409  2287.65394826]
(run_backprop_4dvar pid=47464)  [ 2437.37962688  2437.16858009  2436.96895017]
(run_backprop_4dvar pid=47464)  [ 2485.9109797   2485.66969438  2485.44152364]
(run_backprop_4dvar pid=47464)  [ 2480.27370066  2480.06406582  2479.86579528]
(run_backprop_4dvar pid=47464)  [ 2525.47289062  252

(run_backprop_4dvar pid=47464) [[ 2185.76344632  2185.76220934  2185.76138058]
(run_backprop_4dvar pid=47464)  [ 2147.38308621  2147.38189117  2147.38109051]
(run_backprop_4dvar pid=47464)  [ 2106.00091869  2105.99988287  2105.99918887]
(run_backprop_4dvar pid=47464)  [ 2055.50673415  2055.50567973  2055.50497328]
(run_backprop_4dvar pid=47464)  [ 2182.44557716  2182.44457381  2182.44390158]
(run_backprop_4dvar pid=47464)  [ 2189.74934497  2189.74839095  2189.74775176]
(run_backprop_4dvar pid=47464)  [ 2170.02212383  2170.02105417  2170.02033751]
(run_backprop_4dvar pid=47464)  [ 2179.0795107   2179.07850011  2179.07782303]
(run_backprop_4dvar pid=47464)  [ 2198.50098339  2198.49996401  2198.49928103]
(run_backprop_4dvar pid=47464)  [ 2344.83110991  2344.83006616  2344.82936686]
(run_backprop_4dvar pid=47464)  [ 2382.83560619  2382.83444346  2382.83366445]
(run_backprop_4dvar pid=47464)  [ 2371.45863486  2371.45763153  2371.45695931]
(run_backprop_4dvar pid=47464)  [ 2397.50334269  239

(run_backprop_4dvar pid=47464)  [7177.69224802 7177.69224802 7177.69224802]] [repeated 164x across cluster]
(run_backprop_4dvar pid=47464)  [6529.7679324  6529.7679324  6529.7679324 ] [repeated 16x across cluster]
(run_backprop_4dvar pid=47464) [[ 2220.0924038   2220.09240372  2220.09240368]
(run_backprop_4dvar pid=47464)  [ 2200.99311505  2200.99311496  2200.99311492]
(run_backprop_4dvar pid=47464)  [ 2164.84605349  2164.84605341  2164.84605337]
(run_backprop_4dvar pid=47464)  [ 2121.82250624  2121.82250616  2121.82250612]
(run_backprop_4dvar pid=47464)  [ 2253.67336267  2253.67336259  2253.67336255]
(run_backprop_4dvar pid=47464)  [ 2267.66257267  2267.66257259  2267.66257256]
(run_backprop_4dvar pid=47464)  [ 2259.49207266  2259.49207257  2259.49207253]
(run_backprop_4dvar pid=47464)  [ 2280.20963081  2280.20963072  2280.20963068]
(run_backprop_4dvar pid=47464)  [ 2328.31045662  2328.31045653  2328.31045648]
(run_backprop_4dvar pid=47464)  [ 2480.12647677  2480.12647668  2480.126476

(run_backprop_4dvar pid=47464) [[ 2197.75295393  2197.75295393  2197.75295393]
(run_backprop_4dvar pid=47464)  [ 2166.10209851  2166.10209851  2166.10209851]
(run_backprop_4dvar pid=47464)  [ 2125.87640295  2125.87640295  2125.87640295]
(run_backprop_4dvar pid=47464)  [ 2077.15864906  2077.15864906  2077.15864906]
(run_backprop_4dvar pid=47464)  [ 2205.27193909  2205.27193909  2205.27193909]
(run_backprop_4dvar pid=47464)  [ 2214.09396487  2214.09396487  2214.09396487]
(run_backprop_4dvar pid=47464)  [ 2197.68391437  2197.68391437  2197.68391437]
(run_backprop_4dvar pid=47464)  [ 2209.49899966  2209.49899966  2209.49899966]
(run_backprop_4dvar pid=47464)  [ 2236.87118453  2236.87118453  2236.87118453]
(run_backprop_4dvar pid=47464)  [ 2384.19208499  2384.19208499  2384.19208499]
(run_backprop_4dvar pid=47464)  [ 2426.22088966  2426.22088966  2426.22088966]
(run_backprop_4dvar pid=47464)  [ 2417.0023308   2417.0023308   2417.0023308 ]
(run_backprop_4dvar pid=47464)  [ 2451.22834563  245

...

(run_backprop_4dvar pid=47464) [[ 2213.62998413  2213.62995716  2213.62994126]
(run_backprop_4dvar pid=47464)  [ 2190.85163269  2190.85160556  2190.85158955]
(run_backprop_4dvar pid=47464)  [ 2153.27548436  2153.27545947  2153.2754448 ]
(run_backprop_4dvar pid=47464)  [ 2108.26217973  2108.26215398  2108.26213879]
(run_backprop_4dvar pid=47464)  [ 2238.7150116   2238.71498727  2238.71497292]
(run_backprop_4dvar pid=47464)  [ 2250.81741587  2250.81739215  2250.81737816]
(run_backprop_4dvar pid=47464)  [ 2239.84504246  2239.84501617  2239.84500067]
(run_backprop_4dvar pid=47464)  [ 2257.29423908  2257.29421353  2257.29419847]
(run_backprop_4dvar pid=47464)  [ 2298.36929501  2298.36926754  2298.36925134]
(run_backprop_4dvar pid=47464)  [ 2448.26883273  2448.26880528  2448.2687891 ]
(run_backprop_4dvar pid=47464)  [ 2498.24756606  2498.24753462  2498.24751608]
(run_backprop_4dvar pid=47464)  [ 2493.7044202   2493.70439278  2493.70437661]
(run_backprop_4dvar pid=47464)  [ 2541.33462091  254

(run_backprop_4dvar pid=47464) [[ 2192.22017213  2192.19358497  2192.17259227]
(run_backprop_4dvar pid=47464)  [ 2157.40796974  2157.38205031  2157.36158451]
(run_backprop_4dvar pid=47464)  [ 2116.56383754  2116.54107156  2116.52309607]
(run_backprop_4dvar pid=47464)  [ 2066.93301218  2066.90980914  2066.8914885 ]
(run_backprop_4dvar pid=47464)  [ 2194.37439939  2194.35242055  2194.33506612]
(run_backprop_4dvar pid=47464)  [ 2202.39555648  2202.37454461  2202.35795384]
(run_backprop_4dvar pid=47464)  [ 2184.32507033  2184.30160995  2184.28308567]
(run_backprop_4dvar pid=47464)  [ 2194.70534972  2194.68312205  2194.66557093]
(run_backprop_4dvar pid=47464)  [ 2218.11239932  2218.08962337  2218.07163958]
(run_backprop_4dvar pid=47464)  [ 2364.89189625  2364.86880685  2364.85057523]
(run_backprop_4dvar pid=47464)  [ 2404.96871445  2404.94280177  2404.92234166]
(run_backprop_4dvar pid=47464)  [ 2394.542997    2394.52069509  2394.50308546]
(run_backprop_4dvar pid=47464)  [ 2424.56829204  242

(run_backprop_4dvar pid=47464) [[ 2181.54819402  2181.54819402  2181.54819402]
(run_backprop_4dvar pid=47464)  [ 2140.90150611  2140.90150611  2140.90150611]
(run_backprop_4dvar pid=47464)  [ 2099.22056043  2099.22056043  2099.22056043]
(run_backprop_4dvar pid=47464)  [ 2048.24985128  2048.24985128  2048.24985128]
(run_backprop_4dvar pid=47464)  [ 2175.28286723  2175.28286723  2175.28286723]
(run_backprop_4dvar pid=47464)  [ 2182.21450237  2182.21450237  2182.21450237]
(run_backprop_4dvar pid=47464)  [ 2161.72768853  2161.72768853  2161.72768853]
(run_backprop_4dvar pid=47464)  [ 2170.20184263  2170.20184263  2170.20184263]
(run_backprop_4dvar pid=47464)  [ 2187.39967845  2187.39967845  2187.39967845]
(run_backprop_4dvar pid=47464)  [ 2333.61041665  2333.61041665  2333.61041665]
(run_backprop_4dvar pid=47464)  [ 2370.23597532  2370.23597532  2370.23597532]
(run_backprop_4dvar pid=47464)  [ 2358.70603631  2358.70603631  2358.70603631]
(run_backprop_4dvar pid=47464)  [ 2383.18685095  238

2024-03-13 00:17:35,895	INFO tune.py:1047 -- Total run time: 3976.73 seconds (3976.67 seconds for the tuning loop).


CPU times: user 3min 16s, sys: 40.1 s, total: 3min 56s
Wall time: 2h 19min 3s


In [8]:
full_results_df = pd.concat(all_results_df_list)
full_results_df.to_csv('./pyqg_jax_raytune_sgdopt_werrors_v10_25epochs_system_dims_16_20_24_32.csv')